In [1]:
from torchrl.envs import GymEnv

env = GymEnv("Pendulum-v1")


/home/ezipe/git/learn-torchrl/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(
/home/ezipe/git/learn-torchrl/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward_space` for environment variables or `env.get_wrapper_attr('reward_space')` that will search the reminding wrappers.
  logger.warn(


In [12]:
from torchrl.modules import ProbabilisticActor, MLP, NormalParamWrapper, TanhNormal
from tensordict.nn import TensorDictModule
from torchrl.collectors import SyncDataCollector
from torchrl.modules.tensordict_module.actors import ValueOperator
from torch import nn

from learning_torchrl import Config

conf = Config()

# TODO not sure if this is quite correct
n_obs = env.observation_space.shape[0]
policy = ProbabilisticActor(
    TensorDictModule(
        NormalParamWrapper(
            MLP(
                in_features=n_obs,
                out_features=env.action_space.shape[0] * 2, # mean and std
                num_cells=[conf.policy_hidden] * conf.policy_layers,
            ),
        ),
        in_keys=["obs"],
        out_keys=["loc", "scale"],
    ),
    in_keys=["loc", "scale"],
    distribution_class=TanhNormal,
)

In [20]:
td.from_dict({"obs": torch.randn(10, n_obs)})

TensorDict(
    fields={
        obs: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([10, 3]),
    device=None,
    is_shared=False)

In [36]:
import torch
from tensordict import TensorDict as td

nn = MLP(
    in_features=n_obs,
    out_features=env.action_space.shape[0] * 2,
    num_cells=[conf.policy_hidden] * conf.policy_layers,
)

# nn(torch.randn(10, n_obs))

td_nn = TensorDictModule(NormalParamWrapper(nn), in_keys=['obs'], out_keys=['loc', 'scale'])

# td_nn(td({'obs': torch.randn(10, n_obs)}, batch_size=10)) # now it can take in a tensordict
# td_nn(torch.randn(10, n_obs))

policy = ProbabilisticActor(td_nn, in_keys=['loc', 'scale'], distribution_class=torch.distributions.Normal) # now it samples from a distribution paramereterized by the output of the nn
policy(td({"obs": torch.randn(10, n_obs)}, batch_size=10))['action']

tensor([[ 0.0134],
        [ 0.0108],
        [-0.0117],
        [ 0.0272],
        [ 0.0065],
        [ 0.0346],
        [ 0.0302],
        [-0.0175],
        [ 0.0219],
        [-0.0223]], grad_fn=<SplitBackward0>)